# 컨테이너 기본 개념 및 실습

> **실습 환경**: 이 노트북은 GitHub Codespaces에서 실행하도록 구성되어 있습니다. Docker, Azure CLI 등 필요한 도구가 미리 설치되어 있습니다.

이 노트북에서는 컨테이너의 기본 개념, 컨테이너 이미지 빌드 및 실행, 그리고 Azure Container Registry(ACR)로 이미지를 푸시하는 방법을 실습합니다.

## 1. 컨테이너란 무엇인가?
컨테이너는 애플리케이션과 그 실행 환경을 하나의 패키지로 묶어 어디서나 일관되게 실행할 수 있도록 해주는 기술입니다.

### 컨테이너의 주요 이점
- **이식성**: 어디서나 동일하게 실행 가능
- **경량화**: 가상머신보다 리소스 사용이 적음
- **빠른 배포 및 확장**: 이미지 기반으로 빠르게 배포 및 확장 가능
- **격리성**: 애플리케이션 간 충돌 방지

### VM vs 컨테이너 아키텍처 상세 비교

가상머신(VM)과 컨테이너는 모두 애플리케이션을 격리하여 실행하는 기술이지만, 구조와 동작 방식에 차이가 있습니다.

#### 1. VM 아키텍처

아래는 VM 기반 아키텍처의 구조를 나타낸 다이어그램입니다.

```
[앱1]         [앱2]
   |              |
[게스트 OS1]  [게스트 OS2]
   \           /
   [하이퍼바이저]
       |
   [호스트 OS]
       |
   [하드웨어]
```

- 각 VM은 별도의 게스트 OS를 포함하므로 무겁고, 리소스 사용이 많습니다.
- 하이퍼바이저가 하드웨어와 게스트 OS 사이에서 가상화를 담당합니다.

#### 2. 컨테이너 아키텍처

컨테이너 기반 아키텍처의 구조는 다음과 같습니다.

```
   [앱1]      [앱2]
     |           |
[컨테이너1] [컨테이너2]
   |         |
[컨테이너 엔진]
     |
[호스트 OS]
     |
[하드웨어]
```

- 컨테이너는 호스트 OS의 커널을 공유하며, 별도의 OS가 필요하지 않아 경량화되어 있습니다.
- 컨테이너 엔진(예: Docker)이 컨테이너의 실행과 관리를 담당합니다.

#### 3. 비교 요약

| 항목         | VM                                 | 컨테이너                        |
|--------------|-------------------------------------|---------------------------------|
| OS           | 각 VM마다 별도 게스트 OS 필요        | 호스트 OS 커널 공유             |
| 리소스 사용  | 무거움, 오버헤드 큼                  | 경량, 오버헤드 적음             |
| 시작 속도    | 느림                                | 빠름                            |
| 이식성       | 제한적                               | 매우 높음                       |
| 격리 수준    | 강력한 격리(보안성 높음)             | 프로세스 수준 격리              |

### 컨테이너 아키텍처
컨테이너는 호스트 OS의 커널을 공유하며, 각 컨테이너는 독립적으로 실행됩니다. 대표적인 컨테이너 엔진으로는 Docker가 있습니다.

## 2. 실습 환경 설정

> **참고:** 이 실습 가이드는 [Spring 공식 Docker 가이드](https://spring.io/guides/gs/spring-boot-docker)를 기반으로 작성되었습니다. 보다 자세한 내용이나 최신 정보를 원한다면 해당 링크를 참고해 주세요.

> **GitHub Codespaces 환경**: Maven, Java, Docker, Python, Azure CLI 등이 이미 설치되어 있으므로 별도 설치가 필요하지 않습니다.

### 2.1 Python 가상환경 생성

노트북 작업을 위한 Python 가상환경을 먼저 생성합니다. 이를 통해 패키지 의존성을 격리하여 관리할 수 있습니다.

#### 터미널에서 가상환경 생성

GitHub Codespaces 터미널에서 아래 명령어를 순서대로 실행하세요.

```bash
# Python 가상환경 생성
python3.12 -m venv .venv

# 가상환경 활성화
source .venv/bin/activate

# 가상환경 활성화 확인
which python
python --version
```

> **참고**: 가상환경이 활성화되면 터미널 프롬프트 앞에 `(.venv)`가 표시됩니다.

#### 필요한 Python 패키지 설치

가상환경이 활성화된 상태에서 필요한 패키지를 설치합니다.

In [40]:
# 설치된 패키지 확인
%pip list

Package                                    Version
------------------------------------------ -----------
antlr4-python3-runtime                     4.13.2
applicationinsights                        0.11.10
appnope                                    0.1.4
argcomplete                                3.5.3
asttokens                                  3.0.0
azure-appconfiguration                     1.7.1
azure-batch                                15.0.0b2
azure-cli                                  2.77.0
azure-cli-core                             2.77.0
azure-cli-telemetry                        1.1.0
azure-common                               1.1.28
azure-core                                 1.35.1
azure-cosmos                               3.2.0
azure-data-tables                          12.4.0
azure-datalake-store                       1.0.1
azure-keyvault-administration              4.4.0
azure-keyvault-certificates                4.7.0
azure-keyvault-keys                        4.11.0


In [41]:
# pip 업그레이드
%pip install --upgrade pip

# 필요한 패키지 설치
%pip install azure-cli azure-mgmt-containerregistry azure-mgmt-containerservice requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


> **참고**: 가상환경은 이후 모든 실습에서 활성화된 상태로 유지하세요. 새 터미널을 열 때마다 `source .venv/bin/activate` 명령으로 다시 활성화해야 합니다.

### 2.2 환경 확인

GitHub Codespaces에 설치된 도구들의 버전을 확인합니다.

In [42]:
%%bash
# Maven 버전 확인
mvn -v

# Java 버전 확인
java -version

# Docker 버전 확인
docker --version

# Azure CLI 버전 확인
az --version

Apache Maven 3.9.9 (8e8579a9e76f7d015ee5ec7bfcdc97d260186937)
Maven home: /opt/homebrew/Cellar/maven/3.9.9/libexec
Java version: 23.0.2, vendor: Homebrew, runtime: /opt/homebrew/Cellar/openjdk/23.0.2/libexec/openjdk.jdk/Contents/Home
Default locale: en_KR, platform encoding: UTF-8
OS name: "mac os x", version: "26.0.1", arch: "aarch64", family: "mac"
/3.9.9/libexec
Java version: 23.0.2, vendor: Homebrew, runtime: /opt/homebrew/Cellar/openjdk/23.0.2/libexec/openjdk.jdk/Contents/Home
Default locale: en_KR, platform encoding: UTF-8
OS name: "mac os x", version: "26.0.1", arch: "aarch64", family: "mac"


openjdk version "21.0.7" 2025-04-15 LTS
OpenJDK Runtime Environment Temurin-21.0.7+6 (build 21.0.7+6-LTS)
OpenJDK 64-Bit Server VM Temurin-21.0.7+6 (build 21.0.7+6-LTS, mixed mode, sharing)
-LTS)
OpenJDK 64-Bit Server VM Temurin-21.0.7+6 (build 21.0.7+6-LTS, mixed mode, sharing)


Docker version 28.0.4, build b8034c0
azure-cli                         2.77.0

core                              2.77.0
telemetry                          1.1.0

Extensions:
ai-examples                        0.2.5
application-insights               1.2.3
azure-firewall                     1.2.3
containerapp                     1.2.0b3
ml                                2.39.0

Dependencies:
msal                            1.34.0b1
azure-mgmt-resource               23.3.0

Python location '/Users/junwoojeong/GitHub/aks-mini-labs/.venv/bin/python'
Config directory '/Users/junwoojeong/.azure'
Extensions directory '/Users/junwoojeong/.azure/cliextensions'

Python (Darwin) 3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.3.19.1)]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.


## 3. Spring Boot 애플리케이션 준비

이 섹션에서는 이미 준비된 Spring Boot 애플리케이션을 확인하고, 소스코드부터 컨테이너 이미지 빌드, 실행까지의 전체 과정을 단계별로 실습합니다.

> 💡 **참고**: 이 레포지토리에는 이미 완성된 Spring Boot 프로젝트(`springboot-docker-demo`)가 포함되어 있습니다. 새로 생성할 필요 없이 기존 프로젝트를 사용하여 실습을 진행합니다.

### 3.1 프로젝트 디렉터리 확인

먼저 프로젝트 디렉터리가 존재하는지 확인합니다.

In [44]:
%%bash
# 프로젝트 디렉터리 확인
if [ -d "springboot-docker-demo" ]; then
    echo "✓ springboot-docker-demo 디렉터리가 존재합니다."
    ls -la springboot-docker-demo/
else
    echo "✗ springboot-docker-demo 디렉터리를 찾을 수 없습니다."
    exit 1
fi

✓ springboot-docker-demo 디렉터리가 존재합니다.
total 16
drwxr-xr-x@ 6 junwoojeong  staff   192 Oct 11 13:43 .
drwxr-xr-x  9 junwoojeong  staff   288 Oct 11 13:28 ..
-rw-r--r--@ 1 junwoojeong  staff   480 Oct 11 13:47 Dockerfile
-rw-r--r--@ 1 junwoojeong  staff  1377 Oct 11 13:30 pom.xml
drwxr-xr-x@ 4 junwoojeong  staff   128 Oct 11 13:28 src
drwxr-xr-x@ 8 junwoojeong  staff   256 Oct 11 13:35 target


### 3.2 프로젝트 구조 확인

Spring Boot 프로젝트의 디렉터리 구조를 확인합니다.

In [48]:
%%bash
cd springboot-docker-demo
echo "=== 프로젝트 구조 ==="
echo ""
echo "📁 springboot-docker-demo/"
echo "  ├── pom.xml"
echo "  ├── Dockerfile"
echo "  ├── src/"
echo "  │   ├── main/"
echo "  │   │   ├── java/com/example/springbootdocker/"
echo "  │   │   │   └── App.java"
echo "  │   │   └── resources/"
echo "  │   └── test/"
echo "  │       └── java/"
echo "  └── target/"
echo "      └── demo-0.0.1-SNAPSHOT.jar (빌드 후 생성)"
echo ""
echo "=== 주요 파일 확인 ==="
ls -lh pom.xml Dockerfile src/main/java/com/example/springbootdocker/App.java 2>/dev/null || echo "일부 파일을 찾을 수 없습니다."

=== 프로젝트 구조 ===

📁 springboot-docker-demo/
  ├── pom.xml


  ├── Dockerfile
  ├── src/
  │   ├── main/
  │   │   ├── java/com/example/springbootdocker/
  │   │   │   └── App.java
  │   │   └── resources/
  │   └── test/
  │       └── java/
  └── target/
      └── demo-0.0.1-SNAPSHOT.jar (빌드 후 생성)

=== 주요 파일 확인 ===
-rw-r--r--@ 1 junwoojeong  staff   480B Oct 11 13:47 Dockerfile
-rw-r--r--@ 1 junwoojeong  staff   1.3K Oct 11 13:30 pom.xml
-rw-r--r--@ 1 junwoojeong  staff   555B Oct 11 13:31 src/main/java/com/example/springbootdocker/App.java
mple/springbootdocker/
  │   │   │   └── App.java
  │   │   └── resources/
  │   └── test/
  │       └── java/
  └── target/
      └── demo-0.0.1-SNAPSHOT.jar (빌드 후 생성)

=== 주요 파일 확인 ===
-rw-r--r--@ 1 junwoojeong  staff   480B Oct 11 13:47 Dockerfile
-rw-r--r--@ 1 junwoojeong  staff   1.3K Oct 11 13:30 pom.xml
-rw-r--r--@ 1 junwoojeong  staff   555B Oct 11 13:31 src/main/java/com/example/springbootdocker/App.java


### 3.3 pom.xml 확인

Maven 빌드 설정 파일인 `pom.xml`의 내용을 확인합니다. Spring Boot 3.2.5와 Java 17을 사용합니다.

In [49]:
%%bash
cat springboot-docker-demo/pom.xml

<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>
    <groupId>com.example</groupId>
    <artifactId>demo</artifactId>
    <version>0.0.1-SNAPSHOT</version>
    <packaging>jar</packaging>
    <name>demo</name>
    <description>Demo project for Spring Boot</description>
    <parent>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-parent</artifactId>
        <version>3.2.5</version>
        <relativePath/>
    </parent>
    <properties>
        <java.version>17</java.version>
    </properties>
    <dependencies>
        <dependency>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-starter-web</artifactId>
        </dependency>
        <dependency>
            <groupId>org.springframework.boot</groupId>


### 3.4 Application.java 확인

Spring Boot 애플리케이션의 메인 클래스를 확인합니다. 이 애플리케이션은 루트 경로(`/`)에서 간단한 메시지를 반환하는 REST API를 제공합니다.

In [50]:
%%bash
cat springboot-docker-demo/src/main/java/com/example/springbootdocker/App.java

package com.example.springbootdocker;

import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RestController;

@SpringBootApplication
@RestController
public class App {
    @GetMapping("/")
    public String home() {
        return "Hello Docker World from GitHub Codespaces!";
    }
    public static void main(String[] args) {
        SpringApplication.run(App.class, args);
    }
}.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RestController;

@SpringBootApplication
@RestController
public class App {
    @GetMapping("/")
    public String home() {
        return "Hello Docker World from GitHub Codespaces!";
    }
    public static void main(String[] args) {
        SpringApplication.run(App.class, args

## 4. 컨테이너 이미지 빌드 및 테스트

### 4.1 Dockerfile 작성

프로젝트 루트 디렉터리(`springboot-docker-demo`)에 아래 내용으로 `Dockerfile`을 생성하세요.

> **플랫폼 중립성**: 이 Dockerfile은 멀티 스테이지 빌드를 사용하며, `eclipse-temurin` 공식 이미지는 AMD64(Intel/AMD), ARM64(Apple Silicon) 등 여러 아키텍처를 자동으로 지원합니다. Docker가 실행 중인 플랫폼에 맞는 이미지를 자동으로 선택합니다.

In [ ]:
#### Dockerfile 내용 예시

```dockerfile
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-17 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:17-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
```

아래 셀을 실행하면 이 내용으로 Dockerfile이 자동 생성됩니다.

In [51]:
%%bash
cat > springboot-docker-demo/Dockerfile << 'EOF'
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-17 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:17-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
EOF

echo "✅ Dockerfile 생성 완료"
cat springboot-docker-demo/Dockerfile

✅ Dockerfile 생성 완료
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-17 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:17-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-17 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:17-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]


### 4.2 Docker 이미지 빌드

GitHub Codespaces에는 Docker가 이미 설치되어 있습니다. 아래 명령어로 컨테이너 이미지를 빌드하세요.

> **멀티 스테이지 빌드**: Dockerfile 내부에서 Maven 빌드가 자동으로 실행되므로, 별도로 JAR 파일을 빌드할 필요가 없습니다.

> **중요**: AKS는 AMD64 아키텍처를 사용하므로, M1/M2 Mac에서도 `--platform linux/amd64` 옵션을 사용하여 호환되는 이미지를 빌드합니다.

In [70]:
%%bash
cd springboot-docker-demo
docker build --platform linux/amd64 -t myapp:latest .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 556B 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/maven:3.9-eclipse-temurin-17
rom Dockerfile
#1 transferring dockerfile: 556B 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/maven:3.9-eclipse-temurin-17
#2 ...

#3 [auth] library/eclipse-temurin:pull token for registry-1.docker.io
#3 DONE 0.0s

#4 [auth] library/maven:pull token for registry-1.docker.io
#4 DONE 0.0s

#5 [internal] load metadata for docker.io/library/eclipse-temurin:17-jre
#5 ...

#2 [internal] load metadata for docker.io/library/maven:3.9-eclipse-temurin-17
#2 DONE 2.3s

#5 [internal] load metadata for docker.io/library/eclipse-temurin:17-jre
#5 DONE 2.6s

#6 [internal] load .dockerignore
#6 transferring context: 2B done
#6 DONE 0.0s

#7 [internal] load build context
#7 transferring context: 1.20kB done
#7 DONE 0.0s

#8 [sta

### 4.3 빌드된 이미지 확인

빌드가 완료되면 이미지 목록을 확인할 수 있습니다.

In [71]:
!docker images

REPOSITORY                                                                                      TAG                     IMAGE ID       CREATED          SIZE
myapp                                                                                           latest                  0e8da66474af   24 seconds ago   112MB
myacr1760160175.azurecr.io/myapp                                                                latest                  554bb896c728   5 hours ago      434MB
myacr1760159684.azurecr.io/myapp                                                                latest                  554bb896c728   5 hours ago      434MB
myacr1760169422.azurecr.io/myapp                                                                latest                  a228b4deedeb   5 hours ago      434MB
vsc-agentic-ai-labs-148d0cd601910a405c36fdff6a266cae0cad68b667bdbf001a7f3983b559f3d3-features   latest                  8a642fe8091e   3 days ago       3.87GB
crflyoy4n4dll42.azurecr.io/agent-service            

## 5. 컨테이너 실행 실습

빌드한 이미지를 기반으로 컨테이너를 실행하고 테스트합니다.

> **GitHub Codespaces**: Codespaces에서는 포트 포워딩이 자동으로 설정되어, 실행 중인 컨테이너의 포트에 브라우저로 접근할 수 있습니다.

In [54]:
!docker ps

CONTAINER ID   IMAGE          COMMAND               CREATED       STATUS       PORTS                    NAMES
6b08b8a2da36   554bb896c728   "java -jar app.jar"   3 hours ago   Up 3 hours   0.0.0.0:8080->8080/tcp   myapp


In [55]:
!docker run -d -p 8080:8080 --name myapp myapp:latest

docker: Error response from daemon: Conflict. The container name "/myapp" is already in use by container "6b08b8a2da367f37a7b53236d64b25e49f3525e32312b97060e92b31d5eac51e". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


### 애플리케이션 테스트

GitHub Codespaces에서는 포트 포워딩이 자동으로 설정됩니다. VS Code 하단의 "PORTS" 탭에서 8080 포트를 확인하고 브라우저로 접근할 수 있습니다.

또는 아래 명령어로 터미널에서 테스트할 수 있습니다.

In [56]:
!curl http://localhost:8080

Hello Docker World from GitHub Codespaces!

## 6. Azure Container Registry(ACR) 생성 및 이미지 푸시

Azure에 컨테이너 이미지를 저장하려면 ACR을 사용합니다. 이 섹션에서는 ACR을 생성하고 빌드한 이미지를 푸시하는 전체 과정을 실습합니다.

### 6.1 Azure CLI 로그인

GitHub Codespaces에는 Azure CLI가 이미 설치되어 있습니다. 먼저 Azure에 로그인합니다.

In [62]:
import subprocess
import json

# Azure 로그인 (항상 새로 로그인)
print("Azure 로그인 중...")
subprocess.run(["az", "login"], check=True)
print("✅ Azure 로그인 완료\n")

# 모든 구독 목록 조회
accounts_result = subprocess.run(
    ["az", "account", "list"],
    capture_output=True,
    text=True,
    check=True
)

accounts = json.loads(accounts_result.stdout)

# 현재 기본 구독 표시
current = next((acc for acc in accounts if acc.get('isDefault')), None)
if current:
    print(f"\n현재 기본 구독: {current['name']}")
    print(f"\n💡 다른 구독을 사용하려면 터미널에서 다음 명령을 실행하세요:")
    print(f"   az account set --subscription \"<구독 이름 또는 ID>\"")

Azure 로그인 중...


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "2e8663e8-e15f-4dd9-b617-0cfc4f82adca",
    "id": "8627ae60-01d3-4a2d-9c33-89dea54cd4b4",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Visual Studio Enterprise Subscription",
    "state": "Enabled",
    "tenantDefaultDomain": "joonoojeonggmail.onmicrosoft.com",
    "tenantDisplayName": "기본 디렉터리",
    "tenantId": "2e8663e8-e15f-4dd9-b617-0cfc4f82adca",
    "user": {
      "name": "joonoo.jeong@gmail.com",
      "type": "user"
    }
  }
]
✅ Azure 로그인 완료


현재 기본 구독: Visual Studio Enterprise Subscription

💡 다른 구독을 사용하려면 터미널에서 다음 명령을 실행하세요:
   az account set --subscription "<구독 이름 또는 ID>"


### 6.2 Azure 리소스 그룹 생성

ACR을 생성하기 전에 리소스 그룹을 만들어야 합니다. 리소스 그룹은 관련된 Azure 리소스를 논리적으로 묶어주는 컨테이너 역할을 합니다.

In [63]:
%%bash
# 변수 설정 (원하는 이름으로 변경 가능)
RESOURCE_GROUP="aks-mini-labs-rg"
LOCATION="koreacentral"

# 리소스 그룹 생성
az group create \
  --name $RESOURCE_GROUP \
  --location $LOCATION

echo ""
echo "✅ 리소스 그룹 생성 완료: $RESOURCE_GROUP"

/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


{
  "id": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/myResourceGroup",
  "location": "koreacentral",
  "managedBy": null,
  "name": "myResourceGroup",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
ocation": "koreacentral",
  "managedBy": null,
  "name": "myResourceGroup",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}

✅ 리소스 그룹 생성 완료: myResourceGroup


### 6.3 ACR(Azure Container Registry) 생성

이제 컨테이너 이미지를 저장할 ACR을 생성합니다.

> **참고**: ACR 이름은 전역적으로 고유해야 하며, 5-50자의 영숫자만 사용 가능합니다.

In [64]:
import subprocess
import time

# 변수 설정
RESOURCE_GROUP = "myResourceGroup"
ACR_NAME = f"myacr{int(time.time())}"  # 타임스탬프를 추가하여 고유한 이름 생성
SKU = "Basic"

# ACR 생성
result = subprocess.run([
    "az", "acr", "create",
    "--resource-group", RESOURCE_GROUP,
    "--name", ACR_NAME,
    "--sku", SKU,
    "--admin-enabled", "true"
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ ACR 생성 완료!")
print(f"   ACR 이름: {ACR_NAME}")
print(f"   로그인 서버: {ACR_NAME}.azurecr.io")
print("")
print(f"⚠️ 다음 셀에서 사용할 수 있도록 ACR_NAME 변수가 설정되었습니다: {ACR_NAME}")

{
  "adminUserEnabled": true,
  "anonymousPullEnabled": false,
  "autoGeneratedDomainNameLabelScope": "Unsecure",
  "creationDate": "2025-10-11T07:57:05.061039+00:00",
  "dataEndpointEnabled": false,
  "dataEndpointHostNames": [],
  "encryption": {
    "keyVaultProperties": null,
    "status": "disabled"
  },
  "id": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/myResourceGroup/providers/Microsoft.ContainerRegistry/registries/myacr1760169422",
  "identity": null,
  "location": "koreacentral",
  "loginServer": "myacr1760169422.azurecr.io",
  "metadataSearch": "Disabled",
  "name": "myacr1760169422",
  "networkRuleBypassOptions": "AzureServices",
  "networkRuleSet": null,
  "policies": {
    "azureAdAuthenticationAsArmPolicy": {
      "status": "enabled"
    },
    "exportPolicy": {
      "status": "enabled"
    },
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2025-10-11T07:57:17.56191

### 6.4 ACR 정보 확인

생성된 ACR의 상세 정보를 확인합니다.

In [72]:
import subprocess

# 위 셀에서 생성된 ACR_NAME 변수 사용
result = subprocess.run([
    "az", "acr", "show",
    "--name", ACR_NAME,
    "--query", "{Name:name, LoginServer:loginServer, SKU:sku.name, Location:location}",
    "--output", "table"
], capture_output=True, text=True)

print(result.stdout)

Name             LoginServer                 SKU    Location
---------------  --------------------------  -----  ------------
myacr1760169422  myacr1760169422.azurecr.io  Basic  koreacentral



### 6.5 ACR에 로그인

Docker가 ACR에 이미지를 푸시할 수 있도록 ACR에 로그인합니다.

In [73]:
import subprocess

# ACR에 로그인
result = subprocess.run([
    "az", "acr", "login",
    "--name", ACR_NAME
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ ACR 로그인 성공!")

Login Succeeded


✅ ACR 로그인 성공!


### 6.6 이미지 태깅

ACR로 이미지를 푸시하기 전에 ACR 주소로 태깅해야 합니다.

In [74]:
import subprocess

# ACR 로그인 서버 주소 생성
ACR_LOGIN_SERVER = f"{ACR_NAME}.azurecr.io"

# 이미지 태깅
result = subprocess.run([
    "docker", "tag",
    "myapp:latest",
    f"{ACR_LOGIN_SERVER}/myapp:latest"
], capture_output=True, text=True)

print(result.stdout)
print(f"✅ 이미지 태깅 완료: {ACR_LOGIN_SERVER}/myapp:latest")


✅ 이미지 태깅 완료: myacr1760169422.azurecr.io/myapp:latest


### 6.7 이미지 푸시

태깅된 이미지를 ACR로 푸시합니다.

In [75]:
import subprocess

# ACR 로그인 서버 주소 생성
ACR_LOGIN_SERVER = f"{ACR_NAME}.azurecr.io"

# 이미지 푸시
result = subprocess.run([
    "docker", "push",
    f"{ACR_LOGIN_SERVER}/myapp:latest"
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ 이미지 푸시 완료!")

The push refers to repository [myacr1760169422.azurecr.io/myapp]
075338da2d04: Waiting
f35cff1a101a: Waiting
3826740eb44e: Waiting
6b4a5d6f61a4: Waiting
4b3ffd8ccb52: Waiting
4545082253cb: Waiting
8a17a4ba6a83: Waiting
9712caec413d: Waiting
c36a596532b6: Waiting
3826740eb44e: Waiting
6b4a5d6f61a4: Waiting
4b3ffd8ccb52: Waiting
075338da2d04: Waiting
f35cff1a101a: Waiting
8a17a4ba6a83: Waiting
9712caec413d: Waiting
c36a596532b6: Waiting
4545082253cb: Waiting
4545082253cb: Waiting
8a17a4ba6a83: Waiting
075338da2d04: Waiting
f35cff1a101a: Waiting
6b4a5d6f61a4: Waiting
9712caec413d: Pushed
3826740eb44e: Pushed
6b4a5d6f61a4: Pushed
4545082253cb: Pushed
f35cff1a101a: Pushed
075338da2d04: Pushed
c36a596532b6: Pushed
4b3ffd8ccb52: Pushed
8a17a4ba6a83: Pushed
latest: digest: sha256:0e8da66474af3aa52a2a37d70d6490a77ede95fd04389d689b8d155d0b942554 size: 856


✅ 이미지 푸시 완료!


### 6.8 ACR 이미지 목록 확인

ACR에 푸시된 이미지를 확인합니다.

In [76]:
import subprocess

# ACR에 푸시된 이미지 목록 확인
result = subprocess.run([
    "az", "acr", "repository", "list",
    "--name", ACR_NAME,
    "--output", "table"
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ ACR에 저장된 이미지 목록 확인 완료!")

Result
--------
myapp


✅ ACR에 저장된 이미지 목록 확인 완료!


## 7. 정리

이 노트북에서는 컨테이너의 기본 개념부터 실제 Spring Boot 애플리케이션을 컨테이너 이미지로 빌드하고, 실행 및 Azure Container Registry(ACR)로 푸시하는 전체 과정을 GitHub Codespaces 환경에서 실습했습니다.

### 실습한 내용 요약
- ✅ Python 가상환경 생성 및 패키지 설치
- ✅ 컨테이너 개념 및 아키텍처 이해
- ✅ Spring Boot 애플리케이션 작성
- ✅ 플랫폼 중립적 Dockerfile 작성 및 이미지 빌드
- ✅ 컨테이너 실행 및 테스트
- ✅ Azure 리소스 그룹 및 ACR 생성
- ✅ Azure Container Registry에 이미지 푸시


> **다음 단계**: `02-aks-hands-on.ipynb`에서 ACR에 푸시한 이미지를 AKS(Azure Kubernetes Service) 클러스터에 배포하는 방법을 실습합니다.